In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/new_master_dataset.csv")
df

Mounted at /content/drive


,mag,path,filename,class,slide_id,tumor_type
0,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-001.png,benign,22549CD,A
1,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-002.png,benign,22549CD,A
2,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-003.png,benign,22549CD,A
3,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-004.png,benign,22549CD,A
4,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-005.png,benign,22549CD,A
...,...,...,...,...,...,...
7904,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-031.png,malignant,15704,PC
7905,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-032.png,malignant,15704,PC
7906,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-033.png,malignant,15704,PC
7907,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-034.png,malignant,15704,PC


In [2]:
groupd_df = df.groupby("tumor_type")

# You can then perform various operations on the grouped data, such as getting the count of each tumor type
tumor_type_counts = groupd_df.size()
tumor_type_counts

tumor_type
A      444
DC    3451
F     1014
LC     626
MC     792
PC     560
PT     453
TA     569
dtype: int64

In [3]:
# df_train_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/train_df_100.csv")
# df_test_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/test_df_100.csv")
# df_val_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/val_df_100.csv")

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torchvision.transforms as transforms
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import os
import torch
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define constants
data_dirs = ["TA", "PT", "PC", "MC", "LC", "F", "DC", "A"]
data_root = "/content/drive/MyDrive/Breast Cancer Project/IW/100"  # Replace with the root directory of your data
train_split = 0.7

# Create a list to store the paths and labels of all images
all_data = []

# Populate the list with paths and labels
for label, folder in enumerate(data_dirs):
    folder_path = os.path.join(data_root, folder)
    image_files = os.listdir(folder_path)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        all_data.append((image_path, label))

# Split data into training and testing sets
train_data, test_data = train_test_split(all_data, train_size=train_split, shuffle=True, random_state=42)

# Define custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')  # Open image and convert to RGB mode
        if self.transform:
            img = self.transform(img)
        label_tensor = torch.tensor(label, dtype=torch.long)  # Convert label to tensor
        return img, label_tensor

# Image preprocessing with augmentation for training
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(90),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()
])

# Image preprocessing without augmentation for testing and validation
test_val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create custom datasets
train_dataset = CustomDataset(train_data, transform=train_transform)
test_dataset = CustomDataset(test_data, transform=test_val_transform)

# DataLoaders for batching and shuffling
batch_size = 25 # Define the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("facebook/deit-tiny-patch16-224")
model = AutoModelForImageClassification.from_pretrained("facebook/deit-tiny-patch16-224")

# Define optimizer and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, verbose=True)

# Define loss function
criterion = nn.CrossEntropyLoss()

# Move the model to the appropriate device
model.to(device)

# Train the model
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/23.0M [00:00<?, ?B/s]

...

Epoch 11/15, Train Loss: 0.2852, Train Accuracy: 89.70%, Val Loss: 0.5080, Val Accuracy: 83.04%


Epoch 12/15, Train Loss: 0.1889, Train Accuracy: 92.51%, Val Loss: 0.4590, Val Accuracy: 85.28%


Epoch 13/15, Train Loss: 0.1960, Train Accuracy: 92.38%, Val Loss: 0.4585, Val Accuracy: 85.28%


Epoch 14/15, Train Loss: 0.1650, Train Accuracy: 93.61%, Val Loss: 0.3721, Val Accuracy: 87.52%


Epoch 15/15, Train Loss: 0.1986, Train Accuracy: 92.10%, Val Loss: 0.5136, Val Accuracy: 83.84%
Test Accuracy: 83.84%


In [5]:
from sklearn.metrics import classification_report

# Test the model
model.eval()
test_correct = 0
test_total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print classification report
print("Classification Report:")
report = classification_report(all_labels, all_predicted, digits=3)
print(report)

Test Accuracy: 83.84%
Classification Report:
              precision    recall  f1-score   support

           0      0.527     1.000     0.690        39
           1      0.720     0.947     0.818        38
           2      0.927     0.704     0.800        54
           3      0.941     0.800     0.865        80
           4      0.828     0.522     0.640        46
           5      0.960     0.640     0.768        75
           6      0.876     0.938     0.906       257
           7      0.895     0.944     0.919        36

    accuracy                          0.838       625
   macro avg      0.834     0.812     0.801       625
weighted avg      0.865     0.838     0.837       625



In [6]:
# Train the model
num_epochs = 30
for epoch in range(15, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 16/30, Train Loss: 0.1374, Train Accuracy: 95.12%, Val Loss: 0.3276, Val Accuracy: 88.16%


Epoch 17/30, Train Loss: 0.1531, Train Accuracy: 94.64%, Val Loss: 0.5121, Val Accuracy: 82.88%


Epoch 18/30, Train Loss: 0.1536, Train Accuracy: 93.34%, Val Loss: 0.4070, Val Accuracy: 86.88%


Epoch 19/30, Train Loss: 0.1029, Train Accuracy: 95.40%, Val Loss: 0.3136, Val Accuracy: 90.08%


Epoch 20/30, Train Loss: 0.1222, Train Accuracy: 94.78%, Val Loss: 0.3352, Val Accuracy: 88.48%


...

Epoch 26/30, Train Loss: 0.1338, Train Accuracy: 94.92%, Val Loss: 0.3763, Val Accuracy: 88.80%


Epoch 27/30, Train Loss: 0.0636, Train Accuracy: 97.46%, Val Loss: 0.2808, Val Accuracy: 89.76%


Epoch 28/30, Train Loss: 0.0457, Train Accuracy: 98.28%, Val Loss: 0.2641, Val Accuracy: 90.88%


Epoch 29/30, Train Loss: 0.0397, Train Accuracy: 98.49%, Val Loss: 0.2628, Val Accuracy: 91.36%


Epoch 30/30, Train Loss: 0.0371, Train Accuracy: 98.01%, Val Loss: 0.2662, Val Accuracy: 91.52%
Test Accuracy: 91.52%


In [7]:
from sklearn.metrics import classification_report

# Test the model
model.eval()
test_correct = 0
test_total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print classification report
print("Classification Report:")
report = classification_report(all_labels, all_predicted, digits=3)
print(report)

Test Accuracy: 91.52%
Classification Report:
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        39
           1      0.917     0.868     0.892        38
           2      0.920     0.852     0.885        54
           3      0.971     0.838     0.899        80
           4      0.854     0.761     0.805        46
           5      0.911     0.960     0.935        75
           6      0.892     0.961     0.925       257
           7      0.971     0.917     0.943        36

    accuracy                          0.915       625
   macro avg      0.929     0.895     0.910       625
weighted avg      0.917     0.915     0.914       625



In [8]:
# Train the model
num_epochs = 45
for epoch in range(30, num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()
        # Ensure the input tensor is passed correctly
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * labels.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        progress_bar.set_postfix({'Loss': train_loss / total, 'Accuracy': 100 * correct / total})

    train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct / total

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(test_loader.dataset)
    val_accuracy = 100 * correct / total

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Adjust learning rate
    scheduler.step(val_loss)

# Test the model
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

Epoch 31/45, Train Loss: 0.0398, Train Accuracy: 97.87%, Val Loss: 0.2577, Val Accuracy: 91.84%


Epoch 32/45, Train Loss: 0.0342, Train Accuracy: 98.21%, Val Loss: 0.2525, Val Accuracy: 92.00%


Epoch 33/45, Train Loss: 0.0336, Train Accuracy: 98.49%, Val Loss: 0.2572, Val Accuracy: 92.16%


Epoch 34/45, Train Loss: 0.0335, Train Accuracy: 98.08%, Val Loss: 0.2509, Val Accuracy: 92.32%


Epoch 35/45, Train Loss: 0.0335, Train Accuracy: 98.08%, Val Loss: 0.2493, Val Accuracy: 92.64%


...

Epoch 41/45, Train Loss: 0.0303, Train Accuracy: 98.35%, Val Loss: 0.2545, Val Accuracy: 92.32%


Epoch 42/45, Train Loss: 0.0322, Train Accuracy: 98.01%, Val Loss: 0.2528, Val Accuracy: 92.48%


Epoch 43/45, Train Loss: 0.0303, Train Accuracy: 98.42%, Val Loss: 0.2527, Val Accuracy: 92.32%


Epoch 44/45, Train Loss: 0.0299, Train Accuracy: 98.49%, Val Loss: 0.2527, Val Accuracy: 92.32%


Epoch 45/45, Train Loss: 0.0288, Train Accuracy: 98.49%, Val Loss: 0.2528, Val Accuracy: 92.32%
Test Accuracy: 92.32%


In [9]:
from sklearn.metrics import classification_report

# Test the model
model.eval()
test_correct = 0
test_total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU
        outputs = model(images).logits
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * test_correct / test_total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print classification report
print("Classification Report:")
report = classification_report(all_labels, all_predicted, digits=3)
print(report)

Test Accuracy: 92.32%
Classification Report:
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        39
           1      0.917     0.868     0.892        38
           2      0.926     0.926     0.926        54
           3      0.985     0.825     0.898        80
           4      0.833     0.761     0.795        46
           5      0.911     0.960     0.935        75
           6      0.908     0.965     0.936       257
           7      0.971     0.944     0.958        36

    accuracy                          0.923       625
   macro avg      0.932     0.906     0.917       625
weighted avg      0.924     0.923     0.922       625

